# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 39 new papers today
          12 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/11 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.13270


extracting tarball to tmp_2212.13270... done.
Retrieving document from  https://arxiv.org/e-print/2212.13289


/tmp/ipykernel_2093/4030337529.py:34: LatexWarning: 2212.13289 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2212.13363


...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.13818


extracting tarball to tmp_2212.13818...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.13903


extracting tarball to tmp_2212.13903... done.
Retrieving document from  https://arxiv.org/e-print/2212.13914


/tmp/ipykernel_2093/4030337529.py:34: LatexWarning: 2212.13914 did not run properly
invalid header
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13270-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13270) | **Satellite Content and Halo Mass of Galaxy Clusters: Comparison between  Red-Sequence and Halo-based Optical Cluster Finders**  |
|| Jesse B. Golden-Marx, et al. -- incl., <mark>Hekun Li</mark>, <mark>Jun Zhang</mark> |
|*Appeared on*| *2022-12-29*|
|*Comments*| *17 pages, 12 figures, 6 tables*|
|**Abstract**| Cluster cosmology depends critically on how the optical clusters are selected from imaging surveys. We compare the conditional luminosity function (CLF) and weak lensing halo masses between two different cluster samples at fixed richness, detected within the same volume ($0.1{<}z{<}0.34$) using the red-sequence and halo-based methods. After calibrating our CLF deprojection method against mock galaxy samples, we measure the 3D CLFs by cross-correlating clusters with SDSS photometric galaxies. As expected, the CLFs of the red-sequence and halo-based finders exhibit redder and bluer populations, respectively. We also find significant shape discrepancies between the two CLFs at the faint end, where the red-sequence clusters show a strong deficit of faint galaxies but a bump at $M_r{\sim}-20.5$, while the halo-based clusters host an increasing number of blue satellites. By comparing the subsamples of clusters that have a match between the two catalogues to those without matches, we discover that the CLF shape depends sensitively on the cluster centroiding. However, the average weak lensing halo mass between the matched and non-matched clusters are roughly consistent with each other in either cluster sample. Since the colour preferences of the two cluster finders are almost orthogonal, such a consistency indicates that the scatter in the mass-richness relation of either cluster sample is close to random. Therefore, while the choice of how optical clusters are identified impacts the satellite content, our result suggests that it should not introduce strong systematics biases in cluster cosmology. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13363-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13363) | **$α$-attractor inflation: Models and Predictions**  |
|| Sukannya Bhattacharya, Koushik Dutta, Mayukh R. Gangopadhyay, <mark>Anshuman Maharana</mark> |
|*Appeared on*| *2022-12-29*|
|*Comments*| *7 pages, 6 figures*|
|**Abstract**| The $\alpha$-attractor models are some of the most interesting models of inflation from the point of view of upcoming observations in cosmology and are also attractive from the point of view of supergravity. We confront representative models of exponential and polynomial $\alpha$-attractors with the latest cosmological data (Planck' 18+BICEP2/Keck array) to obtain predictions and best-fit values of model parameters. The analysis is done by making use of ModeChord and CosmoMC plugged together via PolyChord. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13400-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13400) | **LAMOST medium-resolution spectroscopic survey of binarity and exotic  star (LAMOST-MRS-B): Observation strategy and target selection**  |
|| <mark>Jiao Li</mark>, et al. -- incl., <mark>Jiang-Dan Li</mark>, <mark>Yan-Jun Guo</mark>, <mark>Chao Liu</mark>, <mark>Li-Fang Li</mark>, <mark>Bo Zhang</mark>, <mark>Jia-Ming Liu</mark>, <mark>Hao-Tong Zhang</mark> |
|*Appeared on*| *2022-12-29*|
|*Comments*| **|
|**Abstract**| LAMOST-MRS-B is one of the sub-surveys of LAMOST medium-resolution (R~7500) spectroscopic survey. It aims at studying the statistical properties (e.g., binary fraction, orbital period distribution, mass ratio distribution) of binary stars and exotic stars. We intend to observe about 30000 stars (10 mag <= G <= 14.5 mag) with at least 10 visits in five years. We first planned to observe 25 plates around the galactic plane in 2018. Then the plates were reduced to 12 in 2019 because of the limitation of observation. At the same time, two new plates located at the high galactic latitude were added to explore binary properties influenced by the different environments. In this survey project, we set the identified exotic and low-metallicity stars with the highest observation priorities. For the rest of the selected stars, we gave higher priority to the relatively brighter stars in order to obtain high-quality spectra as many as possible. Spectra of 49129 stars have been obtained in LAMOST-MRS-B field and released in DR8, of which 28828 and 3375 stars have been visited more than twice and ten times with SNR >= 10, respectively. Most of the sources are B-, A-, and F-type stars with 0.6 < [Fe/H] < 0.4 dex. We also obtain 347 identified variable and exotic stars and about 250 stars with [Fe/H] < 1 dex. We measure radial velocities (RVs) by using 892233 spectra of the stars. The uncertainties of RV achieve about 1 km/s and 10 km/s1 for 95% of late- and early-type stars, respectively. The datasets presented in this paper are available at 0113.00035.">this http URL |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13735-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13735) | **Interactions between the jet and disk wind in a nearby radio  intermediate quasar III Zw 2**  |
|| Ailing Wang, et al. -- incl., <mark>Shaoguang Guo</mark>, <mark>Heino Falcke</mark>, <mark>Yingkang Zhang</mark> |
|*Appeared on*| *2022-12-29*|
|*Comments*| *accepted by ApJ*|
|**Abstract**| Disk winds and jets are ubiquitous in active galactic nuclei (AGN), and how these two components interact remains an open question. We study the radio properties of a radio-intermediate quasar III Zw 2. We detect two jet knots J1 and J2 on parsec scales, which move at a mildly apparent superluminal speed of $1.35\,c$. Two $\gamma$-ray flares were detected in III Zw 2 in 2009--2010, corresponding to the primary radio flare in late 2009 and the secondary radio flare in early 2010. The primary 2009 flare was found to be associated with the ejection of J2. The secondary 2010 flare occurred at a distance of $\sim$0.3 parsec from the central engine, probably resulting from the collision of the jet with the accretion disk wind. The variability characteristics of III Zw 2 (periodic radio flares, unstable periodicity, multiple quasi-periodic signals and possible harmonic relations between them) can be explained by the global instabilities of the accretion disk. These instabilities originating from the outer part of the warped disk propagate inwards and can lead to modulation of the accretion rate and consequent jet ejection. At the same time, the wobbling of the outer disk may also lead to oscillations of the boundary between the disk wind and the jet tunnel, resulting in changes in the jet-wind collision site. III Zw 2 is one of the few cases observed with jet-wind interactions, and the study in this paper is of general interest for gaining insight into the dynamic processes in the nuclear regions of AGN. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13818-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13818) | **Adaptive Optics system of the Evanescent Wave Coronagraph (EvWaCo):  optimised phase plate and DM characterisation**  |
|| Anthony Berdeu, et al. -- incl., <mark>Matthew Ridsdill-Smith</mark>, <mark>Maud Langlois</mark> |
|*Appeared on*| *2022-12-29*|
|*Comments*| **|
|**Abstract**| The Evanescent Wave Coronagraph (EvWaCo) is an achromatic coronagraph mask with adjustable size over the spectral domain [600nm, 900nm] that will be installed at the Thai National Observatory. We present in this work the development of a bench to characterise its Extreme Adaptive Optics system (XAO) comprising a DM192 ALPAO deformable mirror (DM) and a 15x15 Shack-Hartmann wavefront sensor (SH-WFS). In this bench, the turbulence is simulated using a rotating phase plate in a pupil plane. In general, such components are designed using a randomly generated phase screen. Such single realisation does not necessarily provide the wanted structure function. We present a solution to design the printed pattern to ensure that the beam sees a strict and controlled Kolmogorov statistics with the correct 2D structure function. This is essential to control the experimental conditions in order to compare the bench results with the numerical simulations and predictions. This bench is further used to deeply characterise the full 27 mm pupil of the ALPAO DM using a 54x54 ALPAO SH-WFS. We measure the average shape of its influence functions as well as the influence function of each single actuator to study their dispersion. We study the linearity of the actuator amplitude with the command as well as the linearity of the influence function profile. We also study the actuator offsets as well as the membrane shape at 0-command. This knowledge is critical to get a forward model of the DM for the XAO control loop. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13903-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13903) | **A Gap No More: Mechanism for Non-Nuclear Energy to Fill in the Black  Hole Mass Gap**  |
|| <mark>Joshua Ziegler</mark>, Katherine Freese |
|*Appeared on*| *2022-12-29*|
|*Comments*| *34 pages, 7 figures*|
|**Abstract**| Standard stellar evolution models predict that black holes in the range of approximately $50 - 140 M_\odot$ should not exist directly from stellar evolution. This gap appears because stars with masses between 100 and 240 $M_\odot$ are expected to undergo a pair instability supernova and leave behind no remnant, or a pulsational pair instability supernova and leave behind a remnant much smaller than their initial stellar mass. However, black holes have been discovered by the LIGO/Virgo collaboration within this mass range. In previous work (arXiv:2010.00254), we used the stellar evolution code MESA to show that the addition of non-nuclear energy (such as from annihilation of dark matter) could alter the evolution of a 180 $M_\odot$ star so that the observed black holes could be produced from isolated stars. In this paper, we extend this analysis to stars of other masses, and find that sufficient amounts of non-nuclear energy can allow any star to avoid pair instability, and could produce a black hole of mass comparable to the initial stellar mass. In addition, we produce examples of the type of black hole initial mass function that can be produced from this mechanism. These illustrative examples suggest that adding non-nuclear energy to stars offers a way to fully close the mass gap. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13914-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13914) | **Running and running of the running of the scalar spectral index in warm  inflation**  |
|| Suratna Das, <mark>Rudnei O. Ramos</mark> |
|*Appeared on*| *2022-12-29*|
|*Comments*| *10 pages, double column, 0 figures*|
|**Abstract**| Next generation of cosmological observations are expected to improve the measurements of several quantities connected to the primordial inflation in the early Universe. These quantities include for example improved measurements for the spectral index of the scalar curvature of the primordial power spectrum and to also bring a better understanding on the scaling dependence of the primordial spectrum. This includes the running of the tilt and, possibly, also the running of the running. In this paper, we investigate the possibility of generating large runnings in the context of warm inflation. Useful analytical expressions for the runnings are derived in the context of warm inflation in the strong dissipation regime. The results are compared to and discussed for some well motivated primordial inflaton potentials that have recently been of interest in the literature. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error invalid header</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13289-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13289) | **Artificial Intelligence to Enhance Mission Science Output for In-situ  Observations: Dealing with the Sparse Data Challenge**  |
|| M. I. Sitnov, et al. -- incl., <mark>Y.-H. Liu</mark> |
|*Appeared on*| *2022-12-29*|
|*Comments*| *4 pages, 1 figure; Heliophysics 2050 White Paper*|
|**Abstract**| In the Earth's magnetosphere, there are fewer than a dozen dedicated probes beyond low-Earth orbit making in-situ observations at any given time. As a result, we poorly understand its global structure and evolution, the mechanisms of its main activity processes, magnetic storms, and substorms. New Artificial Intelligence (AI) methods, including machine learning, data mining, and data assimilation, as well as new AI-enabled missions will need to be developed to meet this Sparse Data challenge. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

148  publications in the last 7 days.
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers